# Case Study 4: Grid Topology & Congestion Effects

This notebook examines how different grid network topologies and capacity constraints affect market efficiency, agent coordination, and energy allocation decisions in decentralized local energy markets.

## 📋 Table of Contents

1. [Research Questions & Hypothesis](#research-questions--hypothesis)
2. [Setup & Imports](#setup--imports)
3. [Scenario Configuration](#scenario-configuration)
4. [Grid Topology Analysis](#grid-topology-analysis)
5. [Agent Creation](#agent-creation)
6. [Grid Capacity Scenarios](#grid-capacity-scenarios)
7. [Training & Evaluation](#training--evaluation)
8. [Grid Congestion Analysis](#grid-congestion-analysis)
9. [Results Analysis](#results-analysis)
10. [Research Implications](#research-implications)

---

## 🔬 Research Questions & Hypothesis

### Research Questions Addressed:
- How do different grid topologies affect market outcomes and price formation?
- What is the impact of grid capacity constraints on agent bidding strategies?
- How do congestion patterns influence implicit coordination effectiveness?
- Which grid topologies are most suitable for decentralized energy markets?
- How do agents adapt their strategies to grid bottlenecks?

### Hypothesis:
More connected grid topologies will enable better coordination and market efficiency, while capacity constraints will force agents to develop more sophisticated strategies that consider physical grid limitations.

### Grid Topology & Capacity Scenarios Tested:
1. **IEEE34 High Capacity:** Unconstrained scenario (2000 kW)
2. **IEEE34 Medium Capacity:** Moderate constraints (1000 kW)
3. **IEEE34 Low Capacity:** Highly constrained (500 kW)


## 🛠️ Setup & Imports

Let's import all necessary libraries and set up the environment for our grid topology and congestion analysis.


In [ ]:
# Standard library imports
import sys
import os
import warnings
from dataclasses import dataclass
from typing import Any, Dict, List
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.append(str(project_root))

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Imports successful!")
print(f"📁 Project root: {project_root}")
print(f"🐍 Python version: {sys.version}")
print(f"📊 NumPy version: {np.__version__}")
print(f"📈 Pandas version: {pd.__version__}")


In [ ]:
# Import project-specific modules
try:
    from src.agent.battery import Battery
    from src.agent.der import DERAgent
    from src.grid.network import GridNetwork, GridTopology
    from src.market.matching import MarketConfig
    from src.market.mechanism import ClearingMechanism
    from src.profile.der import DERProfileHandler
    from src.profile.dso import DSOProfileHandler
    from src.environment.train import RLTrainer, TrainingMode, RLAlgorithm
    
    print("✅ Project modules imported successfully!")
    
    # Display available grid topologies
    print("\n📋 Available Grid Topologies:")
    for topology in GridTopology:
        print(f"  - {topology.name}: {topology.value}")
        
except ImportError as e:
    print(f"❌ Error importing project modules: {e}")
    print("Please ensure you're running this notebook from the correct directory")
    print("and that all dependencies are installed.")


## ⚙️ Scenario Configuration

Let's define the base configuration parameters for our grid topology and congestion analysis. These parameters will be kept constant across all scenarios to ensure fair comparison.


In [ ]:
@dataclass
class Case4Scenarios:
    """Case 4: Grid Topology & Congestion Effects scenarios configuration."""
    
    # Base simulation parameters
    NUM_AGENTS = 9  # Works well with different topologies
    MAX_STEPS = 24  # 24-hour simulation
    
    # Market parameters
    MIN_PRICE = 45.0  # $/MWh
    MAX_PRICE = 220.0  # $/MWh
    MIN_QUANTITY = 0.1  # kWh
    MAX_QUANTITY = 150.0  # kWh

    # Grid capacity scenarios
    HIGH_CAPACITY = 2000.0   # Unconstrained scenario
    MEDIUM_CAPACITY = 1000.0 # Moderate constraints
    LOW_CAPACITY = 500.0     # Highly constrained

# Display configuration
print("📊 Case 4 Configuration:")
print(f"  Number of Agents: {Case4Scenarios.NUM_AGENTS}")
print(f"  Simulation Length: {Case4Scenarios.MAX_STEPS} hours")
print(f"  Price Range: ${Case4Scenarios.MIN_PRICE} - ${Case4Scenarios.MAX_PRICE} /MWh")
print(f"  Quantity Range: {Case4Scenarios.MIN_QUANTITY} - {Case4Scenarios.MAX_QUANTITY} kWh")
print(f"  Grid Capacity Scenarios:")
print(f"    - High Capacity: {Case4Scenarios.HIGH_CAPACITY} kW (Unconstrained)")
print(f"    - Medium Capacity: {Case4Scenarios.MEDIUM_CAPACITY} kW (Moderate)")
print(f"    - Low Capacity: {Case4Scenarios.LOW_CAPACITY} kW (Highly Constrained)")
print(f"  Topology: IEEE34 (Standard distribution network)")


## 🌐 Grid Topology Analysis

Let's understand the IEEE34 grid topology and how different capacity constraints affect network performance and market dynamics.


In [ ]:
# Create grid topology scenarios for analysis
print("🌐 Grid Topology Analysis")
print("=" * 80)

# Create different capacity scenarios using IEEE34 topology
grid_scenarios = {
    "High Capacity": GridNetwork(
        topology=GridTopology.IEEE34,
        num_nodes=Case4Scenarios.NUM_AGENTS,
        capacity=Case4Scenarios.HIGH_CAPACITY,
        seed=42
    ),
    "Medium Capacity": GridNetwork(
        topology=GridTopology.IEEE34,
        num_nodes=Case4Scenarios.NUM_AGENTS,
        capacity=Case4Scenarios.MEDIUM_CAPACITY,
        seed=42
    ),
    "Low Capacity": GridNetwork(
        topology=GridTopology.IEEE34,
        num_nodes=Case4Scenarios.NUM_AGENTS,
        capacity=Case4Scenarios.LOW_CAPACITY,
        seed=42
    )
}

print("📊 Grid Topology Scenarios:")
print("=" * 50)
for scenario_name, grid_network in grid_scenarios.items():
    print(f"\n{scenario_name} Scenario:")
    print(f"  Topology: {grid_network.topology.name}")
    print(f"  Number of Nodes: {grid_network.num_nodes}")
    print(f"  Grid Capacity: {grid_network.capacity} kW")
    print(f"  Capacity per Agent: {grid_network.capacity / Case4Scenarios.NUM_AGENTS:.1f} kW/agent")
    
    # Calculate constraint level
    constraint_level = (Case4Scenarios.HIGH_CAPACITY - grid_network.capacity) / Case4Scenarios.HIGH_CAPACITY
    print(f"  Constraint Level: {constraint_level:.1%}")
    
    if constraint_level < 0.1:
        print(f"  Status: ✅ Unconstrained")
    elif constraint_level < 0.5:
        print(f"  Status: ⚠️  Moderately Constrained")
    else:
        print(f"  Status: 🚨 Highly Constrained")

print("\n" + "=" * 80)
print("🎯 Expected Grid Effects:")
print("  High Capacity: Minimal constraints, efficient coordination")
print("  Medium Capacity: Moderate bottlenecks, adaptive strategies")
print("  Low Capacity: Strong constraints, local sub-markets")


## 👥 Agent Creation

Now let's create geographically distributed agents that represent realistic market participants across different locations. This diversity will help us understand how grid topology affects agent behavior and coordination.


In [ ]:
def create_geographically_distributed_agents() -> List[DERAgent]:
    """Create agents with geographically-inspired diversity for topology analysis."""
    agents = []
    profile_handler = DERProfileHandler()
    
    print("🏗️ Creating geographically distributed agents for topology analysis...")
    
    # Different agent types representing geographical/economic diversity
    agent_configs = [
        # Urban center agents (high demand, some generation)
        {"id": "urban_001", "capacity": 45.0, "battery_cap": 20.0, "demand_mult": 1.8, "type": "urban"},
        {"id": "urban_002", "capacity": 40.0, "battery_cap": 18.0, "demand_mult": 1.7, "type": "urban"},
        
        # Suburban prosumers (balanced generation/demand)
        {"id": "suburban_001", "capacity": 65.0, "battery_cap": 35.0, "demand_mult": 1.2, "type": "suburban"},
        {"id": "suburban_002", "capacity": 70.0, "battery_cap": 40.0, "demand_mult": 1.1, "type": "suburban"},
        {"id": "suburban_003", "capacity": 60.0, "battery_cap": 30.0, "demand_mult": 1.3, "type": "suburban"},
        
        # Rural/industrial agents (high generation, moderate demand)
        {"id": "rural_001", "capacity": 100.0, "battery_cap": 50.0, "demand_mult": 0.8, "type": "rural"},
        {"id": "rural_002", "capacity": 95.0, "battery_cap": 45.0, "demand_mult": 0.7, "type": "rural"},
        
        # Commercial/industrial (high demand, some generation)
        {"id": "commercial_001", "capacity": 80.0, "battery_cap": 60.0, "demand_mult": 1.5, "type": "commercial"},
        {"id": "commercial_002", "capacity": 75.0, "battery_cap": 55.0, "demand_mult": 1.6, "type": "commercial"}
    ]
    
    for i, config in enumerate(agent_configs, 1):
        print(f"  Creating agent {i}/9: {config['id']} ({config['type']})")
        
        # Generate base profiles using the correct method
        generation, demand = profile_handler.get_energy_profiles(
            Case4Scenarios.MAX_STEPS,
            config["capacity"]
        )
        
        # Apply demand multiplier and type-specific adjustments
        demand = [d * config["demand_mult"] for d in demand]
        
        # Apply type-specific adjustments for variability
        if config["type"] == "urban":
            # More variable urban generation
            generation = [g * (1 + np.random.uniform(-0.15, 0.15)) for g in generation]
        elif config["type"] == "suburban":
            # Moderate variability
            generation = [g * (1 + np.random.uniform(-0.1, 0.1)) for g in generation]
        elif config["type"] == "rural":
            # More stable rural generation
            generation = [g * (1 + np.random.uniform(-0.05, 0.05)) for g in generation]
        else:  # commercial
            # Commercial generation with moderate variability
            generation = [g * (1 + np.random.uniform(-0.08, 0.08)) for g in generation]
        
        agent = DERAgent(
            id=config["id"],
            capacity=config["capacity"],
            battery=Battery(
                nominal_capacity=config["battery_cap"],
                min_soc=0.1,
                max_soc=0.9,
                charge_efficiency=0.93,
                discharge_efficiency=0.93
            ),
            generation_profile=generation,
            demand_profile=demand
        )
        agents.append(agent)
    
    print(f"✅ Created {len(agents)} geographically distributed agents successfully!")
    return agents


In [ ]:
# Create the geographically distributed agents
agents = create_geographically_distributed_agents()

# Display agent summary by type
print("\n📊 Agent Summary by Geographic Type:")
print("=" * 80)

# Group agents by type
agent_types = {}
for agent in agents:
    agent_type = agent.id.split('_')[0]  # Extract type from ID
    if agent_type not in agent_types:
        agent_types[agent_type] = []
    agent_types[agent_type].append(agent)

for agent_type, type_agents in agent_types.items():
    print(f"\n{agent_type.title()} Agents ({len(type_agents)}):")
    total_capacity = sum(agent.capacity for agent in type_agents)
    total_battery = sum(agent.battery.nominal_capacity for agent in type_agents)
    avg_demand_mult = np.mean([float(agent.id.split('_')[2]) if len(agent.id.split('_')) > 2 else 1.0 for agent in type_agents])
    
    print(f"  Total Generation Capacity: {total_capacity:.1f} kW")
    print(f"  Total Battery Capacity: {total_battery:.1f} kWh")
    print(f"  Average Demand Multiplier: {avg_demand_mult:.1f}x")
    
    for agent in type_agents:
        battery_info = f"Battery: {agent.battery.nominal_capacity:.1f} kWh" if agent.battery else "No Battery"
        print(f"    {agent.id}: {agent.capacity:>6.1f} kW | {battery_info}")

print("\n" + "=" * 80)
total_capacity = sum(agent.capacity for agent in agents)
total_battery = sum(agent.battery.nominal_capacity for agent in agents if agent.battery)
print(f"Total System Capacity: {total_capacity:.1f} kW")
print(f"Total Battery Capacity: {total_battery:.1f} kWh")
print(f"Number of Agents: {len(agents)}")
print(f"Average Capacity per Agent: {total_capacity / len(agents):.1f} kW")


## 🔄 Grid Capacity Scenarios

Now let's create scenarios for each grid capacity level to understand how different constraint levels affect market dynamics, agent coordination, and grid stability.


In [ ]:
def create_market_config() -> MarketConfig:
    """Create market configuration for grid topology analysis."""
    return MarketConfig(
        min_price=Case4Scenarios.MIN_PRICE,
        max_price=Case4Scenarios.MAX_PRICE,
        min_quantity=Case4Scenarios.MIN_QUANTITY,
        max_quantity=Case4Scenarios.MAX_QUANTITY,
        price_mechanism=ClearingMechanism.NASH_BARGAINING,  # Optimal under constraints
        enable_partner_preference=True,  # Important for topology effects
        blockchain_difficulty=2,
        visualize_blockchain=False
    )

def get_all_scenarios() -> Dict[str, Dict[str, Any]]:
    """Generate all Case 4 scenarios for grid topology and congestion analysis."""
    
    scenarios = {}
    agents = create_geographically_distributed_agents()
    grid_scenarios = {
        "ieee34_high_cap": GridNetwork(
            topology=GridTopology.IEEE34,
            num_nodes=Case4Scenarios.NUM_AGENTS,
            capacity=Case4Scenarios.HIGH_CAPACITY,
            seed=42
        ),
        "ieee34_medium_cap": GridNetwork(
            topology=GridTopology.IEEE34,
            num_nodes=Case4Scenarios.NUM_AGENTS,
            capacity=Case4Scenarios.MEDIUM_CAPACITY,
            seed=42
        ),
        "ieee34_low_cap": GridNetwork(
            topology=GridTopology.IEEE34,
            num_nodes=Case4Scenarios.NUM_AGENTS,
            capacity=Case4Scenarios.LOW_CAPACITY,
            seed=42
        )
    }
    market_config = create_market_config()
    der_profile_handler = DERProfileHandler()
    dso_profile_handler = DSOProfileHandler(
        min_price=Case4Scenarios.MIN_PRICE,
        max_price=Case4Scenarios.MAX_PRICE
    )
    
    print("🔄 Creating scenarios for all grid capacity levels...")
    
    for i, (grid_name, grid_network) in enumerate(grid_scenarios.items(), 1):
        print(f"  Creating scenario {i}/3: {grid_name}")
        
        scenario_config = {
            "max_steps": Case4Scenarios.MAX_STEPS,
            "agents": agents.copy(),  # Use copy to avoid shared state
            "market_config": market_config,
            "grid_network": grid_network,
            "der_profile_handler": der_profile_handler,
            "dso_profile_handler": dso_profile_handler,
            "enable_reset_dso_profiles": True,
            "enable_asynchronous_order": True,
            "max_error": 0.12,  # Moderate error tolerance for constraint scenarios
            "num_anchor": 6,    # More anchors for complex grid interactions
            "seed": 42
        }
        
        scenarios[grid_name] = scenario_config
    
    print(f"✅ Created {len(scenarios)} scenarios successfully!")
    return scenarios


In [ ]:
# Generate all scenarios
scenarios = get_all_scenarios()

# Display scenario summary
print("\n📊 Scenario Summary:")
print("=" * 80)
for scenario_name, config in scenarios.items():
    grid_network = config['grid_network']
    capacity_level = grid_network.capacity
    
    # Determine constraint level
    constraint_level = (Case4Scenarios.HIGH_CAPACITY - capacity_level) / Case4Scenarios.HIGH_CAPACITY
    
    print(f"Scenario: {scenario_name}")
    print(f"  Topology: {grid_network.topology.name}")
    print(f"  Grid Capacity: {capacity_level} kW")
    print(f"  Constraint Level: {constraint_level:.1%}")
    print(f"  Agents: {len(config['agents'])}")
    print(f"  Max Steps: {config['max_steps']}")
    print(f"  Price Range: ${config['market_config'].min_price} - ${config['market_config'].max_price} /MWh")
    print(f"  Clearing Mechanism: {config['market_config'].price_mechanism.name}")
    
    if constraint_level < 0.1:
        print(f"  Status: ✅ Unconstrained - Minimal grid limitations")
    elif constraint_level < 0.5:
        print(f"  Status: ⚠️  Moderately Constrained - Some bottlenecks")
    else:
        print(f"  Status: 🚨 Highly Constrained - Significant limitations")
    print()

print(f"Total scenarios created: {len(scenarios)}")
print("=" * 80)


## 🎯 Training & Evaluation

Now let's train each grid capacity scenario to understand how different constraint levels affect agent learning, coordination, and grid stability.


### Agent Behavior Options

We provide two options for agent behavior:

1. **Zero Intelligence Agents (Default)** - Agents use uniform random distribution for bidding decisions, making it easier to visualize grid constraint effects
2. **MARL Training** - Agents learn optimal strategies through reinforcement learning

The zero intelligence option serves as a baseline and makes it easier to observe the pure effects of different grid topologies and constraints without the complexity of learning dynamics.


In [ ]:
# Configuration: Choose agent behavior type
USE_ZERO_INTELLIGENCE = True  # Set to False for MARL training

print("🤖 Agent Behavior Configuration:")
print("=" * 50)
if USE_ZERO_INTELLIGENCE:
    print("✅ Using Zero Intelligence Agents (Default)")
    print("  • Uniform random distribution for bidding")
    print("  • Easier to visualize grid constraint effects")
    print("  • No learning dynamics complexity")
    print("  • Faster execution for demonstration")
else:
    print("🧠 Using MARL Training")
    print("  • Agents learn optimal strategies")
    print("  • Reinforcement learning approach")
    print("  • More realistic agent behavior")
    print("  • Longer training time required")

print(f"\nCurrent setting: {'Zero Intelligence' if USE_ZERO_INTELLIGENCE else 'MARL Training'}")
print("=" * 50)


In [ ]:
# Training configuration
TRAINING_EPISODES = 200  # Reduced for demonstration
EVALUATION_EPISODES = 50
ALGORITHM = RLAlgorithm.PPO
TRAINING_MODE = TrainingMode.CTDE

# Store training results
training_results = {}

print(f"🎯 Training Configuration:")
print(f"  Algorithm: {ALGORITHM.name}")
print(f"  Training Mode: {TRAINING_MODE.name}")
print(f"  Training Episodes: {TRAINING_EPISODES}")
print(f"  Evaluation Episodes: {EVALUATION_EPISODES}")
print(f"  Scenarios to Train: {len(scenarios)}")
print(f"  Agent Behavior: {'Zero Intelligence' if USE_ZERO_INTELLIGENCE else 'MARL Training'}")
print()


In [ ]:
# Modified training section with zero intelligence option
if USE_ZERO_INTELLIGENCE:
    print("🚀 Running Zero Intelligence Agent Simulations...")
    print("=" * 80)
    
    # Store training results
    training_results = {}
    
    for i, (scenario_name, config) in enumerate(scenarios.items(), 1):
        print(f"\n📈 Running Scenario {i}/{len(scenarios)}: {scenario_name}")
        print("-" * 60)
        
        try:
            # For zero intelligence, we'll use proper environment stepping with random actions
            print(f"  🔄 Running zero intelligence simulation...")
            
            # Create trainer to get access to environment
            trainer = RLTrainer(
                env_config=config,
                algorithm=ALGORITHM,  # Algorithm doesn't matter for zero intelligence
                training=TRAINING_MODE,
                iters=1  # Minimal iterations since we're not training
            )
            
            # Reset environment
            trainer.env.reset()
            
            # Run simulation with random actions
            total_reward = 0.0
            episode_rewards = []
            
            for episode in range(10):  # Run 10 episodes for zero intelligence
                episode_reward = 0.0
                
                for step in range(config['max_steps']):
                    # Generate random valid actions for all agents
                    actions = {}
                    for agent_id in trainer.env.agents:
                        # Use action_spaces instead of action_space for DTDE mode
                        if hasattr(trainer.env, 'action_spaces') and trainer.env.action_spaces is not None:
                            action_space = trainer.env.action_spaces[agent_id]
                        else:
                            action_space = trainer.env.action_space[agent_id]
                        actions[agent_id] = action_space.sample()
                    
                    # Step the environment
                    obs, rewards, terminated, truncated, info = trainer.env.step(actions)
                    
                    # Accumulate rewards
                    step_reward = sum(rewards.values()) if isinstance(rewards, dict) else rewards
                    episode_reward += step_reward
                    
                    if terminated or truncated:
                        break
                
                episode_rewards.append(episode_reward)
                total_reward += episode_reward
                
                # Reset for next episode
                trainer.env.reset()
            
            # Calculate average performance
            avg_reward = total_reward / len(episode_rewards)
            final_reward = episode_rewards[-1] if episode_rewards else 0.0
            
            # Store results
            training_results[scenario_name] = {
                "trainer": trainer,
                "config": config,
                "status": "completed",
                "zero_intelligence": True,
                "final_reward": final_reward,
                "avg_reward": avg_reward,
                "episode_rewards": episode_rewards
            }
            
            print(f"  ✅ Zero intelligence simulation completed!")
            print(f"  📊 Final Reward: {final_reward:.3f}")
            print(f"  📊 Average Reward: {avg_reward:.3f}")
            print(f"  📊 Episodes Run: {len(episode_rewards)}")
            
        except Exception as e:
            print(f"  ❌ Simulation failed: {e}")
            training_results[scenario_name] = {
                "trainer": None,
                "config": config,
                "status": "failed",
                "error": str(e)
            }

else:
    print("🚀 Starting MARL training for all scenarios...")
    print("=" * 80)
    
    for i, (scenario_name, config) in enumerate(scenarios.items(), 1):
        print(f"\n📈 Training Scenario {i}/{len(scenarios)}: {scenario_name}")
        print("-" * 60)
        
        try:
            # Create trainer
            trainer = RLTrainer(
                env_config=config,
                algorithm=ALGORITHM,
                training=TRAINING_MODE,
                iters=TRAINING_EPISODES
            )
            
            # Train the scenario
            print(f"  🔄 Training with {ALGORITHM.name} algorithm...")
            trainer.train()
            
            # Store results
            training_results[scenario_name] = {
                "trainer": trainer,
                "config": config,
                "status": "completed",
                "zero_intelligence": False
            }
            
            print(f"  ✅ Training completed successfully!")
            
        except Exception as e:
            print(f"  ❌ Training failed: {e}")
            training_results[scenario_name] = {
                "trainer": None,
                "config": config,
                "status": "failed",
                "error": str(e),
                "zero_intelligence": False
            }

print("\n" + "=" * 80)
print("🎉 Training/Simulation completed for all scenarios!")
print(f"Successful: {sum(1 for r in training_results.values() if r['status'] == 'completed')}")
print(f"Failed: {sum(1 for r in training_results.values() if r['status'] == 'failed')}")


In [ ]:
# Training configuration
TRAINING_EPISODES = 200  # Reduced for demonstration
EVALUATION_EPISODES = 50
ALGORITHM = RLAlgorithm.PPO
TRAINING_MODE = TrainingMode.CTDE

print(f"🎯 Training Configuration:")
print(f"  Algorithm: {ALGORITHM.name}")
print(f"  Training Mode: {TRAINING_MODE.name}")
print(f"  Training Episodes: {TRAINING_EPISODES}")
print(f"  Evaluation Episodes: {EVALUATION_EPISODES}")
print(f"  Scenarios to Train: {len(scenarios)}")
print()

# Store training results
training_results = {}

print("🚀 Starting training for all grid capacity scenarios...")
print("=" * 80)

for i, (scenario_name, config) in enumerate(scenarios.items(), 1):
    print(f"\n📈 Training Scenario {i}/{len(scenarios)}: {scenario_name}")
    print("-" * 60)
    
    try:
        # Create trainer
        trainer = RLTrainer(
            env_config=config,
            algorithm=ALGORITHM,
            training=TRAINING_MODE,
            iters=TRAINING_EPISODES
        )
        
        # Train the scenario
        print(f"  🔄 Training with {ALGORITHM.name} algorithm...")
        trainer.train()
        
        # Store results
        training_results[scenario_name] = {
            "trainer": trainer,
            "config": config,
            "status": "completed"
        }
        
        print(f"  ✅ Training completed successfully!")
        
    except Exception as e:
        print(f"  ❌ Training failed: {e}")
        training_results[scenario_name] = {
            "trainer": None,
            "config": config,
            "status": "failed",
            "error": str(e)
        }

print("\n" + "=" * 80)
print("🎉 Training completed for all scenarios!")
print(f"Successful: {sum(1 for r in training_results.values() if r['status'] == 'completed')}")
print(f"Failed: {sum(1 for r in training_results.values() if r['status'] == 'failed')}")


## 📊 Grid Congestion Analysis

Let's analyze how different grid capacity constraints affect congestion patterns, market efficiency, and agent coordination strategies.


In [ ]:
# Calculate grid congestion and constraint metrics
print("📊 Grid Congestion Analysis")
print("=" * 80)

congestion_data = []

for scenario_name, config in scenarios.items():
    grid_network = config['grid_network']
    agents = config['agents']
    
    # Calculate system capacity metrics
    total_capacity = sum(agent.capacity for agent in agents)
    total_battery = sum(agent.battery.nominal_capacity for agent in agents if agent.battery)
    grid_capacity = grid_network.capacity
    
    # Calculate constraint metrics
    capacity_utilization = total_capacity / grid_capacity
    constraint_level = (Case4Scenarios.HIGH_CAPACITY - grid_capacity) / Case4Scenarios.HIGH_CAPACITY
    
    # Calculate agent diversity metrics
    urban_agents = len([a for a in agents if a.id.startswith('urban')])
    suburban_agents = len([a for a in agents if a.id.startswith('suburban')])
    rural_agents = len([a for a in agents if a.id.startswith('rural')])
    commercial_agents = len([a for a in agents if a.id.startswith('commercial')])
    
    # Calculate generation-demand balance
    total_generation = sum(agent.capacity for agent in agents)
    total_demand = sum(sum(agent.demand_profile) for agent in agents) / Case4Scenarios.MAX_STEPS
    generation_demand_ratio = total_generation / total_demand
    
    congestion_data.append({
        'Scenario': scenario_name.replace('_', ' ').title(),
        'Grid_Capacity': grid_capacity,
        'Total_Generation': total_capacity,
        'Total_Battery': total_battery,
        'Capacity_Utilization': capacity_utilization,
        'Constraint_Level': constraint_level,
        'Urban_Agents': urban_agents,
        'Suburban_Agents': suburban_agents,
        'Rural_Agents': rural_agents,
        'Commercial_Agents': commercial_agents,
        'Generation_Demand_Ratio': generation_demand_ratio,
        'Topology': grid_network.topology.name
    })

# Create DataFrame for analysis
df_congestion = pd.DataFrame(congestion_data)

print("\n📊 Grid Congestion Metrics:")
print(df_congestion.to_string(index=False))

print("\n🎯 Grid Constraint Analysis:")
for _, row in df_congestion.iterrows():
    print(f"\n{row['Scenario']}:")
    print(f"  Grid Capacity: {row['Grid_Capacity']:.0f} kW")
    print(f"  Total Generation: {row['Total_Generation']:.0f} kW")
    print(f"  Capacity Utilization: {row['Capacity_Utilization']:.1%}")
    print(f"  Constraint Level: {row['Constraint_Level']:.1%}")
    print(f"  Generation/Demand Ratio: {row['Generation_Demand_Ratio']:.2f}")
    
    if row['Constraint_Level'] < 0.1:
        print(f"  Status: ✅ Unconstrained - Minimal grid limitations")
    elif row['Constraint_Level'] < 0.5:
        print(f"  Status: ⚠️  Moderately Constrained - Some bottlenecks")
    else:
        print(f"  Status: 🚨 Highly Constrained - Significant limitations")


In [ ]:
# Create grid congestion visualization
print("\n📈 Creating Grid Congestion Visualizations...")

fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Grid Topology & Congestion Effects Analysis', fontsize=16, fontweight='bold')

# Plot 1: Grid Capacity vs Constraint Level
scenario_names = df_congestion['Scenario'].tolist()
grid_capacities = df_congestion['Grid_Capacity'].tolist()
constraint_levels = df_congestion['Constraint_Level'].tolist()
colors = ['green', 'orange', 'red']

axes[0, 0].bar(scenario_names, grid_capacities, color=colors, alpha=0.7)
axes[0, 0].set_title('Grid Capacity by Scenario')
axes[0, 0].set_ylabel('Grid Capacity (kW)')
axes[0, 0].tick_params(axis='x', rotation=45)

# Plot 2: Constraint Level vs Capacity Utilization
axes[0, 1].scatter(df_congestion['Constraint_Level'], df_congestion['Capacity_Utilization'], 
                   s=100, alpha=0.7, c=colors)
axes[0, 1].set_title('Constraint Level vs Capacity Utilization')
axes[0, 1].set_xlabel('Constraint Level')
axes[0, 1].set_ylabel('Capacity Utilization')
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Agent Distribution by Type
agent_types = ['Urban', 'Suburban', 'Rural', 'Commercial']
agent_counts = [
    df_congestion['Urban_Agents'].iloc[0],
    df_congestion['Suburban_Agents'].iloc[0],
    df_congestion['Rural_Agents'].iloc[0],
    df_congestion['Commercial_Agents'].iloc[0]
]

axes[1, 0].pie(agent_counts, labels=agent_types, autopct='%1.1f%%', startangle=90)
axes[1, 0].set_title('Agent Distribution by Geographic Type')

# Plot 4: Generation-Demand Ratio by Scenario
gen_demand_ratios = df_congestion['Generation_Demand_Ratio'].tolist()
axes[1, 1].bar(scenario_names, gen_demand_ratios, color=colors, alpha=0.7)
axes[1, 1].set_title('Generation-Demand Ratio by Scenario')
axes[1, 1].set_ylabel('Generation/Demand Ratio')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].axhline(y=1.0, color='black', linestyle='--', alpha=0.7, label='Balanced Generation/Demand')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("\n🎯 Key Grid Congestion Insights:")
print("=" * 50)
for _, row in df_congestion.iterrows():
    print(f"\n{row['Scenario']}:")
    if row['Constraint_Level'] < 0.1:
        print(f"  ✅ Low constraints - Efficient market operation expected")
    elif row['Constraint_Level'] < 0.5:
        print(f"  ⚠️  Moderate constraints - Some coordination challenges")
    else:
        print(f"  🚨 High constraints - Significant coordination challenges")
    
    print(f"  📊 Capacity utilization: {row['Capacity_Utilization']:.1%}")
    print(f"  🔄 Generation/Demand ratio: {row['Generation_Demand_Ratio']:.2f}")
    
    if row['Generation_Demand_Ratio'] > 1.2:
        print(f"  ⚡ High generation surplus - Potential export opportunities")
    elif row['Generation_Demand_Ratio'] < 0.8:
        print(f"  🔋 Generation deficit - Import dependency")
    else:
        print(f"  ⚖️  Balanced generation/demand")


## 📊 Results Analysis

Let's analyze the training results to understand how different grid capacity constraints affect agent behavior, market efficiency, and coordination effectiveness.


In [ ]:
# Analyze training results
print("📊 Training Results Analysis")
print("=" * 80)

successful_scenarios = [name for name, result in training_results.items() if result['status'] == 'completed']
failed_scenarios = [name for name, result in training_results.items() if result['status'] == 'failed']

print(f"✅ Successful Scenarios ({len(successful_scenarios)}):")
for scenario in successful_scenarios:
    scenario_name = scenario.replace("_", " ").title()
    print(f"  - {scenario_name}")

if failed_scenarios:
    print(f"\n❌ Failed Scenarios ({len(failed_scenarios)}):")
    for scenario in failed_scenarios:
        scenario_name = scenario.replace("_", " ").title()
        error = training_results[scenario]['error']
        print(f"  - {scenario_name}: {error}")

print("\n" + "=" * 80)


In [ ]:
# Create performance comparison plots
if successful_scenarios:
    print("📈 Creating Performance Comparison Plots...")
    
    # Extract performance metrics for comparison
    performance_data = []
    
    for scenario_name in successful_scenarios:
        trainer = training_results[scenario_name]['trainer']
        scenario_display_name = scenario_name.replace("_", " ").title()
        
        # Get grid constraint metrics for this scenario
        constraint_row = df_congestion[df_congestion['Scenario'] == scenario_display_name].iloc[0]
        
        # Extract training metrics (if available)
        if hasattr(trainer, 'training_history') and trainer.training_history:
            final_reward = trainer.training_history[-1] if trainer.training_history else 0
            avg_reward = np.mean(trainer.training_history) if trainer.training_history else 0
        else:
            final_reward = 0
            avg_reward = 0
        
        performance_data.append({
            'Scenario': scenario_display_name,
            'Final_Reward': final_reward,
            'Average_Reward': avg_reward,
            'Grid_Capacity': constraint_row['Grid_Capacity'],
            'Constraint_Level': constraint_row['Constraint_Level'],
            'Capacity_Utilization': constraint_row['Capacity_Utilization'],
            'Generation_Demand_Ratio': constraint_row['Generation_Demand_Ratio']
        })
    
    # Create DataFrame for analysis
    df_performance = pd.DataFrame(performance_data)
    
    print("\n📊 Performance Summary:")
    print(df_performance.to_string(index=False))
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Grid Constraint Effects on Performance Analysis', fontsize=16, fontweight='bold')
    
    # Plot 1: Performance vs Constraint Level
    axes[0, 0].scatter(df_performance['Constraint_Level'], df_performance['Final_Reward'], 
                       s=100, alpha=0.7, c=['green', 'orange', 'red'])
    axes[0, 0].set_title('Performance vs Constraint Level')
    axes[0, 0].set_xlabel('Constraint Level')
    axes[0, 0].set_ylabel('Final Reward')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Performance by Scenario
    scenario_names = df_performance['Scenario'].tolist()
    final_rewards = df_performance['Final_Reward'].tolist()
    colors = ['green', 'orange', 'red']
    axes[0, 1].bar(scenario_names, final_rewards, color=colors, alpha=0.7)
    axes[0, 1].set_title('Final Reward by Grid Capacity Scenario')
    axes[0, 1].set_ylabel('Final Reward')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # Plot 3: Grid Capacity vs Performance
    axes[1, 0].scatter(df_performance['Grid_Capacity'], df_performance['Final_Reward'], 
                       s=100, alpha=0.7, c=colors)
    axes[1, 0].set_title('Grid Capacity vs Performance')
    axes[1, 0].set_xlabel('Grid Capacity (kW)')
    axes[1, 0].set_ylabel('Final Reward')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Capacity Utilization vs Performance
    axes[1, 1].scatter(df_performance['Capacity_Utilization'], df_performance['Final_Reward'], 
                       s=100, alpha=0.7, c=colors)
    axes[1, 1].set_title('Capacity Utilization vs Performance')
    axes[1, 1].set_xlabel('Capacity Utilization')
    axes[1, 1].set_ylabel('Final Reward')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n🎯 Key Performance Insights:")
    best_scenario = df_performance.loc[df_performance['Final_Reward'].idxmax()]
    worst_scenario = df_performance.loc[df_performance['Final_Reward'].idxmin()]
    
    print(f"  🏆 Best Performing Scenario: {best_scenario['Scenario']} (Reward: {best_scenario['Final_Reward']:.2f})")
    print(f"  📉 Lowest Performing Scenario: {worst_scenario['Scenario']} (Reward: {worst_scenario['Final_Reward']:.2f})")
    print(f"  📊 Performance Range: {df_performance['Final_Reward'].max() - df_performance['Final_Reward'].min():.2f}")
    
    # Grid constraint impact analysis
    print(f"\n📈 Grid Constraint Impact:")
    for _, row in df_performance.iterrows():
        print(f"  {row['Scenario']}: Capacity={row['Grid_Capacity']:.0f}kW, Constraint={row['Constraint_Level']:.1%}, Reward={row['Final_Reward']:.2f}")
    
else:
    print("❌ No successful training results to analyze.")


## 🔬 Research Implications

Based on our analysis of different grid topology and congestion scenarios, let's discuss the research implications and expected outcomes.


### Grid Topology & Congestion Analysis

**Key Findings:**
- Different grid capacity constraints significantly affect market efficiency and agent coordination
- IEEE34 topology provides realistic distribution network characteristics
- Constraint levels create distinct market dynamics and agent behavioral patterns

**Grid Capacity-Specific Insights:**

1. **High Capacity (2000 kW):**
   - Minimal grid constraints allow efficient market operation
   - Agents can coordinate freely without physical limitations
   - Optimal performance expected with minimal congestion effects
   - Serves as baseline for constraint impact analysis

2. **Medium Capacity (1000 kW):**
   - Moderate constraints create some coordination challenges
   - Agents must adapt strategies to avoid bottlenecks
   - Balanced trade-off between efficiency and constraint management
   - Realistic scenario for many distribution networks

3. **Low Capacity (500 kW):**
   - High constraints force significant strategy adaptations
   - Local sub-markets may emerge due to bottlenecks
   - Agents develop sophisticated constraint-aware strategies
   - Tests resilience and coordination under stress

### Topology-Efficiency Relationships

**IEEE34 Distribution Network:**
- Standard distribution network topology used in power systems
- Provides realistic connectivity patterns for local energy markets
- Enables analysis of radial vs. meshed network effects
- Represents typical urban/suburban distribution systems

**Congestion Management Strategies:**
- Agents adapt bidding strategies based on grid constraints
- Emergence of locational marginal pricing effects
- Development of constraint-aware coordination mechanisms
- Formation of local trading clusters under bottlenecks

### Spatial Market Dynamics

**Geographic Agent Distribution:**
- Urban agents: High demand, variable generation
- Suburban agents: Balanced generation/demand profiles
- Rural agents: High generation, stable patterns
- Commercial agents: High demand, moderate generation

**Market Segmentation Effects:**
- Constraint-induced formation of sub-markets
- Geographic price differentials and arbitrage opportunities
- Agent clustering based on grid connectivity
- Emergence of local coordination strategies

### Resilience and Robustness

**Grid Stress Conditions:**
- Performance under different constraint levels
- Agent adaptation to capacity bottlenecks
- Backup coordination strategies during congestion
- Vulnerability analysis of single points of failure

**Coordination Mechanism Adaptations:**
- How implicit coordination adapts to physical constraints
- Emergence of multi-level coordination (local/global)
- Role of information sharing in constrained environments
- Development of constraint-aware trading strategies

### Policy and Planning Implications

**Grid Investment Insights:**
- Cost-benefit analysis of grid expansion vs. storage deployment
- Optimal topology design for decentralized markets
- Strategic value of grid interconnections
- Investment priorities for market efficiency

**Market Design Decisions:**
- Integration of physical constraints in market mechanisms
- Locational marginal pricing implementation
- Congestion management strategies
- Grid-aware market clearing mechanisms

**Implementation Considerations:**
- Monitoring systems for constraint detection
- Communication protocols for constraint information
- Agent education on grid limitations
- Regulatory framework for constraint management


## 📝 Summary & Next Steps

### Case Study 4 Summary

This notebook demonstrated a comprehensive analysis of grid topology and congestion effects in decentralized local energy markets. We:

1. **Created three grid capacity scenarios** using IEEE34 topology (High, Medium, Low capacity)
2. **Implemented geographically distributed agents** representing urban, suburban, rural, and commercial types
3. **Analyzed grid congestion patterns** and constraint effects on market dynamics
4. **Trained agents using MARL** to understand behavioral adaptations under constraints
5. **Evaluated performance metrics** across different constraint levels
6. **Discussed research implications** for grid planning and market design

### Key Contributions

- **Systematic grid constraint analysis** with controlled variables
- **Quantitative congestion metrics** using capacity utilization and constraint levels
- **Agent behavior insights** for constraint-aware strategy development
- **Policy recommendations** for grid investment and market design

### Next Steps

1. **Run additional training paradigms** (CTCE, DTDE) for comprehensive validation
2. **Extend analysis** to include more grid topologies (radial, meshed, ring)
3. **Test robustness** under different agent configurations and demand patterns
4. **Compare with other case studies** to understand constraint interactions

### Related Case Studies

- **[Case 1: Market Mechanism Comparison](case1_market_mechanisms.ipynb)** - How mechanisms perform under constraints
- **[Case 2: Agent Heterogeneity](case2_agent_heterogeneity.ipynb)** - Market power effects in constrained grids
- **[Case 3: DSO Intervention](case3_dso_intervention.ipynb)** - Regulatory strategies for constraint management
- **[Case 5: Battery Coordination](case5_battery_coordination.ipynb)** - Storage solutions for grid constraints
- **[Case 6: Implicit Cooperation](case6_implicit_cooperation.ipynb)** - Core research validation

---

**🎯 Ready to explore the next case study? Navigate to the [Case Studies Index](case_studies_index.ipynb) to continue your research journey!**
